## Read in Data

In [144]:
# Regression experiments for 4741

# spotify <- read.table(file="Spotify4741.10.26.csv", header=TRUE, sep=",")

# reading in the new file

spotify <- read.table(file="Spotify4741_Pop.csv", header=TRUE, sep=",")

In [145]:
sum(is.na(spotify))

[1] 0

In [146]:
#Read in columns that we only care about
spotify3 <- spotify[, c("acousticness", "danceability", "energy", "liveness", "loudness", "speechiness", "tempo", "valence", "Like.or.not.like")]
#Convert -1 to 0 
spotify3$Like.or.not.like[spotify3$Like.or.not.like == -1] <- 0

In [147]:
sum(is.na(spotify3))

[1] 0

# ROC Curve Selection for Logistic Regression

In [148]:
# Logistic Regression
spotify_logit <- glm(Like.or.not.like~., data = spotify3, family = binomial)
summary(spotify_logit)


Call:
glm(formula = Like.or.not.like ~ ., family = binomial, data = spotify3)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4839  -1.0113  -0.8486   1.2551   1.6978  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)
(Intercept)   2.61272    2.32204   1.125    0.261
acousticness  0.19567    1.21941   0.160    0.873
danceability  1.62677    1.63532   0.995    0.320
energy       -2.72078    1.73653  -1.567    0.117
liveness      0.45445    1.40070   0.324    0.746
loudness      0.18602    0.12486   1.490    0.136
speechiness  -1.55861    2.72743  -0.571    0.568
tempo        -0.00441    0.00753  -0.586    0.558
valence      -1.09143    1.05841  -1.031    0.302

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 232.11  on 170  degrees of freedom
Residual deviance: 225.41  on 162  degrees of freedom
AIC: 243.41

Number of Fisher Scoring iterations: 4


## Logistic Regression

In [125]:
#Production Run 
n=500;
overall = rep(0,n); tprcol = rep(0,n); recommended = rep(0,n); weightedaverage = rep(0,n);
overall1 = rep(0,n); tprcol1 = rep(0,n); recommended1 = rep(0,n); weightedaverage1 = rep(0,n);

set.seed(10)
#Set aside cross validation set
cv <- sample(1:nrow(spotify3),1/10*nrow(spotify3))
cv1 <- spotify3[cv,]
spotify3 <- spotify3[-cv, ]
for (j in 1:n) {
    train_ind <- sample(1:nrow(spotify3), 2/3*nrow(spotify3))
    spotify3_train <- spotify3[train_ind, ]
    spotify3_test <- spotify3[-train_ind, ]

    #Training fit of function 
    spotify3_logit <- glm(Like.or.not.like~., data = spotify3_train, family = binomial)
    #Summary Results
    summary(spotify3_logit)
    #Testing fit for TESTING
    spotify3_pred <- predict(spotify3_logit, spotify3_test, type = "response")

    #fit for CROSSVALIDATION
    spotify3_cv <- predict(spotify3_logit, cv1, type = "response")

    #Sample fit for 0.4 threshold TESTING
    spotify_bin <- as.numeric(spotify3_pred >= 0.4)
    table(spotify_bin, spotify3_test$Like.or.not.like)
    testtable=table(spotify_bin, spotify3_test$Like.or.not.like)
    
    #Sample fit for 0.4 threshold CROSSVALIDATION
    spotify_bin <- as.numeric(spotify3_cv >= 0.4)
    table(spotify_bin, cv1$Like.or.not.like)
    cvtable=table(spotify_bin, cv1$Like.or.not.like)
    
    #Check that table is 2x2 then only do the following for testing
    if (dim(testtable)[1] == 2 && dim(testtable)[2] == 2 && (testtable[1,1]+testtable[2,2])/(testtable[1,1]+testtable[2,2]+testtable[1,2]+testtable[2,1]) != 0 && testtable[2,2]/(sum(testtable[,2])) != 0 && testtable[2,2]/(sum(testtable[2,])) != 0) 
        overall[j] = (testtable[1,1]+testtable[2,2])/(testtable[1,1]+testtable[2,2]+testtable[1,2]+testtable[2,1]);
        tprcol[j] = testtable[2,2]/(sum(testtable[,2]));
        recommended[j] = testtable[2,2]/(sum(testtable[2,]));
        weightedaverage[j] = 0.5*tprcol[j] + 0.5*recommended[j];
    
    # cross validation
    if (dim(cvtable)[1] == 2 && dim(cvtable)[2] == 2 && (cvtable[1,1]+cvtable[2,2])/(cvtable[1,1]+cvtable[2,2]+cvtable[1,2]+cvtable[2,1]) != 0 && cvtable[2,2]/(sum(cvtable[,2])) != 0 && cvtable[2,2]/(sum(cvtable[2,])) != 0 ){    
        #cross validating values
        overall1[j] = (cvtable[1,1]+cvtable[2,2])/(cvtable[1,1]+cvtable[2,2]+cvtable[1,2]+cvtable[2,1]);
        tprcol1[j] = cvtable[2,2]/(sum(cvtable[,2]));
        recommended1[j] = cvtable[2,2]/(sum(cvtable[2,]));
        weightedaverage1[j] = 0.5*tprcol1[j] + 0.5*recommended1[j];
    }
}

In [126]:
print("testing values")
mean(overall);mean(tprcol);mean(recommended);mean(weightedaverage);
print("cv values")
mean(overall1);mean(tprcol1);mean(recommended1);mean(weightedaverage1);

[1] "testing values"


[1] 0.6077829

[1] 0.7792086

[1] 0.5581133

[1] 0.668661

[1] "cv values"


[1] 0.6387931

[1] 0.7287429

[1] 0.6903724

[1] 0.7095576